Lets look at the data

In [ ]:
# !pip install openpyxl

In [ ]:
import pandas as pd
import config

In [ ]:
train_df = pd.read_excel(config.TRAIN_FILE_PATH)
test_df = pd.read_excel(config.TEST_FILE_PATH)

In [ ]:
print("Null data: train")
print(train_df.isnull().sum())

print("Null data: test")
print(test_df.isnull().sum())

In [30]:
print(train_df['Sentence'].str.len().describe())
print(train_df['Entity'].str.len().describe())

count    5943.00000
mean      107.33754
std        64.20659
min        12.00000
25%        64.00000
50%        93.00000
75%       132.00000
max       769.00000
Name: Sentence, dtype: float64
count    5943.000000
mean        7.412418
std         3.259509
min         2.000000
25%         5.000000
50%         7.000000
75%         8.000000
max        31.000000
Name: Entity, dtype: float64


In [31]:
print(test_df['Sentence'].str.len().describe())
print(test_df['Entity'].str.len().describe())

count    1214.000000
mean       94.734761
std        63.549090
min         6.000000
25%        52.000000
50%        81.000000
75%       121.000000
max       583.000000
Name: Sentence, dtype: float64
count    1214.000000
mean        7.639209
std         3.218861
min         3.000000
25%         5.000000
50%         7.000000
75%         9.000000
max        27.000000
Name: Entity, dtype: float64


In [32]:
print(train_df['Sentence'].str.split().str.len().describe())
print(train_df['Entity'].str.split().str.len().describe())

count    5943.000000
mean       19.336867
std        12.342277
min         2.000000
25%        11.000000
50%        17.000000
75%        24.000000
max       143.000000
Name: Sentence, dtype: float64
count    5943.000000
mean        1.120646
std         0.347733
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         3.000000
Name: Entity, dtype: float64


In [33]:
print(test_df['Sentence'].str.split().str.len().describe())
print(test_df['Entity'].str.split().str.len().describe())

count    1214.000000
mean       17.191928
std        11.920604
min         1.000000
25%         9.000000
50%        15.000000
75%        22.000000
max       107.000000
Name: Sentence, dtype: float64
count    1214.000000
mean        1.149918
std         0.377342
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         3.000000
Name: Entity, dtype: float64


In [ ]:
train_df['entity_counters'] = train_df.apply(lambda x: x['Sentence'].count(f"{x['Entity']}"), axis = 1)
train_df['entity_counters'].value_counts()

In [ ]:
test_df['entity_counters'] = test_df.apply(lambda x: x['Sentence'].count(f"{x['Entity']}"), axis = 1)
test_df['entity_counters'].value_counts()

Just small preprocessing will solve this issue. Reducing all to lower will reduce the knowledge we gain from casing of sentences

In [ ]:
train_df.iloc[[668], [1]] = 'Companies'
train_df.iloc[[2346], [1]] = 'Customer Services'
train_df.iloc[[2028], [1]] = 'customer services'
train_df.iloc[[3206], [1]] = 'Advisor'
train_df.iloc[[4555], [1]] = 'Transaction'
train_df.iloc[[3315], [1]] = 'Website'
train_df.iloc[[1527], [1]] = 'pet-hate'
train_df.iloc[[2826], [1]] = 'AXA Insurance'

In [ ]:
train_df['entity_counters'] = train_df.apply(lambda x: x['Sentence'].count(f"{x['Entity']}"), axis = 1)
train_df['entity_counters'].value_counts()

In [ ]:
train_df = train_df[train_df['entity_counters'] != 0]
test_df = test_df[test_df['entity_counters'] != 0]

In [ ]:
train_df['entity_counters'] = train_df.apply(lambda x: x['Sentence'].count(f"{x['Entity']}"), axis = 1)
train_df['entity_counters'].value_counts()

In [ ]:
import re
from tqdm.notebook import tqdm

tqdm.pandas()
train_df['entity_counters'] = train_df.progress_apply(lambda x: len(re.findall(r"\b" + re.escape(x['Entity']) + r"\b", x['Sentence'])), axis = 1)
train_df['entity_counters'].value_counts()

In [ ]:
tqdm.pandas()
test_df['entity_counters'] = test_df.progress_apply(lambda x: len(re.findall(r"\b" + re.escape(x['Entity']) + r"\b", x['Sentence'])), axis = 1)
test_df['entity_counters'].value_counts()

Still some issues remaining. Will have to manually clean them.

In [ ]:
train_df = train_df[train_df['entity_counters'] != 0]
test_df = test_df[test_df['entity_counters'] != 0]

In [ ]:
train_df.to_csv('../data/processed_train.csv')
test_df.to_csv('../data/processed_test.csv')